In [ ]:
import zipfile
import os
import pandas as pd

def unzip_and_combine(zip_dir, file_name, format):
    zip_list = os.listdir(f"./{zip_dir}")
    
    

In [ ]:
data_loc = "/home/seho/data"
zip_data_loc = "/home/seho/data/zip/마이비 카드 이력 정보"
unzip_data_loc = "/home/seho/data/unzip/마이비 카드 이력 정보"

In [ ]:
if os.path.exists(zip_data_loc) == False:
    os.makedirs(zip_data_loc)
    
if os.path.exists(unzip_data_loc) == False:
    os.makedirs(unzip_data_loc)    

In [1]:
import zipfile
import os
import pandas as pd

In [2]:
zip_list = os.listdir(zip_data_loc)

In [3]:
! rm ./MYBI/*.csv

In [4]:
for i, z_file in enumerate(zip_list):
    # zip이 아닌 파일은 Pass
    if os.path.splitext(z_file)[1] != ".zip":
        continue
    print(f"{i}. Extracting {z_file} ...")
    
    with zipfile.ZipFile(f"{zip_data_loc}/{z_file}", 'r') as zip_ref:
        # file size가 0인 경우는 Pass 한다
        file_size = [x for x in zip_ref.filelist if x.filename == "MYBICARDHISTORY.csv"][0].file_size
        if file_size == 0:
            print("File size 0")
            continue
        
        temp = zip_ref.open("MYBICARDHISTORY.csv")
        print("Find MYBICARDHISTORY.csv")
        pd_temp = pd.read_csv(temp, dtype = {"ROUTE_NM" : str})
        pd_temp.columns = [x.lower() for x in pd_temp.columns]
        
        if "Unnamed: 0" in pd_temp.columns:
            pd_temp = pd_temp.drop("Unnamed: 0", axis = 1)
        
        # 컬럼명(header)만 있고 파일이 없는 경우는 Pass
        if pd_temp.shape[0] == 0:
            continue
        min_date = str(pd_temp["collectdate"].min())[2:]
        max_date = str(pd_temp["collectdate"].max())[2:]

        pd_temp.to_csv(f"{unzip_data_loc}/mybicard_{min_date}_{max_date}.csv", index = False)
        print(f"mybicard_{min_date}_{max_date}.csv")
        del pd_temp
        del temp            

0. Extracting 29_BIS_MYBI - 2021-02-09T174836.150.zip ...
Find MYBICARDHISTORY.csv
mybicard_200627_200701.csv
1. Extracting 29_BIS_MYBI (20).zip ...
Find MYBICARDHISTORY.csv
mybicard_200928_201004.csv
2. Extracting 29_BIS_MYBI (88).zip ...
Find MYBICARDHISTORY.csv
mybicard_200723_200727.csv
3. Extracting 29_BIS_MYBI - 2021-02-09T175043.707.zip ...
Find MYBICARDHISTORY.csv
mybicard_200503_200507.csv
4. Extracting 29_BIS_MYBI - 2021-02-09T174926.874.zip ...
Find MYBICARDHISTORY.csv
mybicard_200609_200613.csv
5. Extracting 29_BIS_MYBI (82).zip ...
Find MYBICARDHISTORY.csv
mybicard_200729_200803.csv
6. Extracting 29_BIS_MYBI - 2021-02-09T175012.733.zip ...
Find MYBICARDHISTORY.csv
mybicard_200527_200531.csv
7. Extracting 29_BIS_MYBI - 2021-02-09T174920.708.zip ...
Find MYBICARDHISTORY.csv
mybicard_200615_200621.csv
8. Extracting 29_BIS_MYBI (85).zip ...
Find MYBICARDHISTORY.csv
mybicard_200726_200729.csv
9. Extracting 29_BIS_MYBI - 2021-02-09T175105.121.zip ...
Find MYBICARDHISTORY.csv
myb

#### CSV 파일 하나로 합치기

In [5]:
mybi_csv_gen = (pd.read_csv(f"{unzip_data_loc}/{x}", dtype = {"route_nm" : str}) for x in os.listdir(unzip_data_loc) if os.path.splitext(x)[1] == ".csv")

In [6]:
%%time
mybicard = pd.concat(mybi_csv_gen)

CPU times: user 43.1 s, sys: 3.54 s, total: 46.6 s
Wall time: 46.6 s


In [7]:
import pyarrow.feather as feather

In [8]:
%%time
with open('mybicard.ftr', 'wb') as f:
    feather.write_feather(mybicard, f)

CPU times: user 12.6 s, sys: 2.41 s, total: 15 s
Wall time: 10.6 s


In [10]:
mybicard.shape

(37479749, 15)

In [12]:
mybicard = mybicard.drop_duplicates()